In [1]:
from logging import error
from web3.auto import w3
import json
import requests
import os
from dotenv import load_dotenv
from datetime import datetime as dt

load_dotenv()


Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


True

In [2]:
def init_contract(abi_path:str, contract_address:str):
    """ Initialize a contract given the abi and address """
    with open(abi_path) as json_file:
        abi = json.load(json_file)
    return w3.eth.contract(address=contract_address, abi=abi)



In [3]:
os.getenv("CHARITY_MAKER_ADDRESS")

'0x3729D6a0d46dC56F0758c8Bd5Ba758f7ECBc2E50'

In [4]:
charity_contract = init_contract("CharityMakerABI.json", os.getenv("CHARITY_MAKER_ADDRESS"))


In [5]:
headers = {
    "Content-Type": "application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY"),
}

In [6]:
def convertDataToJSON(content:dict):
    data = {
        "pinataOptions": {"cidVersion": 1},
        "pinataContent": content,
    }
    return json.dumps(data)

def pinJSONtoIPFS(json):
    req = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS", data=json, headers=headers
    )
    ipfs_hash = req.json()["IpfsHash"]
    return ipfs_hash


In [7]:
def toDict(dictToParse):
    # convert any 'AttributeDict' type found to 'dict'
    parsedDict = dict(dictToParse)
    for key, val in parsedDict.items():
        # check for nested dict structures to iterate through
        if  'dict' in str(type(val)).lower():
            parsedDict[key] = toDict(val)
        # convert 'HexBytes' type to 'str'
        elif 'HexBytes' in str(type(val)):
            parsedDict[key] = val.hex()
    return parsedDict

def get_charityEventID_from_URI(event_URI: str):
    charity_event_reg_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"URI": event_URI})
    charity_event_registrations = charity_event_reg_filter.get_all_entries()
    charity_event_registrations_dict = toDict(charity_event_registrations[0])
    return charity_event_registrations_dict['args']['charityEventID']

def register_charity_event(event_name: str, event_recipient: str, funding_goal: int, start_date, end_date):

    # convert string start and end dates to datetime (if they aren't already datetime objects)
    if not isinstance(start_date, dt):
        start_date = dt.strptime(start_date, "%Y/%m/%d")  # 2020/01/26
    if not isinstance(end_date, dt):
        end_date = dt.strptime(end_date, "%Y/%m/%d")

    # check if start_date is in the past and if so make the start_date today instead
    # today = dt.strptime(dt.now(), "%Y/%m/%d")
    today = dt.now()
    if start_date < today:
        start_date = today
    
    # check if end date is after start_date and return error if not
    if start_date > end_date:
        return error(f"Error: End Date {end_date} is before Start Date {start_date}")

    # convert start and end datetimes to to unix timestamps
    start_date = int(start_date.timestamp())
    end_date = int(end_date.timestamp())

    # create charity_info dict
    charity_info = {
        "charityEventName" : event_name,
        # charityEventRecipient: event_recipient,
        "goalAmount" : funding_goal
    }

    # convert charity_info to json_charity_info (json format) and pin to IPFS via pinata api function
    json_charity_info = convertDataToJSON(charity_info)
    ipfs_link = pinJSONtoIPFS(json_charity_info)

    # create charity event in the block chain
    tx_hash = charity_contract.functions.registerCharityEvent(event_recipient, start_date, end_date, ipfs_link).transact({"from": w3.eth.accounts[0]})
    receipt  = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # function registerCharityEvent(
    #     address payable _charityEventAddress,
    #     uint _startDate,
    #     uint _endDate,
    #     string memory _URI

    
    # lookup registerCharityEvent events, filtering by the newly created ipfs_link (URI) to cross reference and return the newly generated charity event id
    charity_event_id = get_charityEventID_from_URI(ipfs_link)

    return charity_event_id


In [8]:
# test register charity
event_id = register_charity_event("Dan's best charity#3", "0x6b36C3CBFcD41C747305e540266AEe5a867f948A", 200, "2020/11/15", "2021/11/15")


In [9]:
event_id

4

In [10]:
def update_charity_event_approval(charity_event_id: int, is_approved: bool):
    # call contract funtion to update approval
    tx_hash = charity_contract.functions.updateCharityEventApproval(charity_event_id, is_approved)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [11]:
# test update_charity_event_approval
update_charity_event_approval(4, True)

AttributeDict({'transactionHash': HexBytes('0x3eddcca4b999c7ee70cc73d5cbd0331b9172e119e590f94f59544f382f44149b'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x0fe844e4199372b39c59f3fc37a89dbfc5a4c96aaf226f9628d191fc7e501738'),
 'blockNumber': 123,
 'from': '0x13C0fAd72f393A9ea42AE90c7a868f509cF13CD6',
 'to': '0x3729D6a0d46dC56F0758c8Bd5Ba758f7ECBc2E50',
 'gasUsed': 43544,
 'cumulativeGasUsed': 43544,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [14]:
charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": 4})
charity_events = charity_event_filter.get_all_entries()
charity_event_dict = toDict(charity_events[0])
charity_event_dict['args']['URI']

# sample http call
# https://gateway.pinata.cloud/ipfs/bafkreiew4vas3g3bi27bx4rblutplsholoaot24lsb3mzmhwwq3ibkunrm

'bafkreiew4vas3g3bi27bx4rblutplsholoaot24lsb3mzmhwwq3ibkunrm'

In [25]:
# get JSON from IPFS
def getJSONfromPinata(ipfs_hash):
#     response = requests.get(f"https://gateway.pinata.cloud/ipfs/{charity_event_dict['args']['URI']}")
#     json_data = json.loads(response.text)
    response = requests.get(f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}")
    return json.loads(response.text)

In [21]:
json_data = getJSONfromIPFS("bafkreiew4vas3g3bi27bx4rblutplsholoaot24lsb3mzmhwwq3ibkunrm")

In [23]:
type(json_data), json_data

(dict, {'charityEventName': "Dan's best charity#3", 'goalAmount': 200})

In [36]:
# charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": charity_event_id})
# charity_events = charity_event_filter.get_all_entries()
# charity_event_dict = toDict(charity_events[0])
charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": 4})
charity_events = charity_event_filter.get_all_entries()
charity_event_dict = toDict(charity_events[0])

charity_event_dict['args']['charityEventAddress']

# charityBook

KeyError: 'charityEventAddress'

In [39]:
# charity_contract.call().charityBook(4)
# charity_contract.methods.charityBook(4).call()
charity_contract.

AttributeError: 'Contract' object has no attribute 'methods'

In [26]:
def get_charity_event(charity_event_id):
    charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": charity_event_id})
    charity_events = charity_event_filter.get_all_entries()
    charity_event_dict = toDict(charity_events[0])

    ipfs_hash = charity_event_dict['args']['URI']
    charity_ipfs_data = getJSONfromPinata(ipfs_hash)

#     ipfs_link = f"ipfs://{}"
#     ipfs_http_gateway_link = f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"
#     charity_event_name = charity_ipfs_data['charityEventName']
#     goal_amount = charity_ipfs_data['goalAmount']
#     charity_event_address = charity_event_dict['args']['charityEventAddress']
#     start_date = charity_event_dict['args']['startDate']
#     end_date = charity_event_dict['args']['endDate']
#     is_approved = 

    charity_event_info = {
        "charityEventName": charity_ipfs_data['charityEventName'],
        "charityEventID": charity_event_id,
        "charityEventAddress": charity_event_dict['args']['charityEventAddress'],
        "startDate": charity_event_dict['args']['startDate'],
        "endDate": charity_event_dict['args']['endDate'],
        "goalAmount": charity_ipfs_data['goalAmount'],
        "isApproved": "HECK YEAH",
        "URI": charity_ipfs_data['URI'],
    }
    # address payable charityEventAddress;
    # uint startDate;
    # uint endDate;
    # bool isApproved;
    # string URI;

    return charity_event_info

In [27]:
event3_info = get_charity_event(4)

KeyError: 'charityEventAddress'